In [ ]:
#@title install edge tts
# https://github.com/rany2/edge-tts/tree/master
!pip install pydub
!pip install googletrans==3.1.0a0
!pip install edge-tts
from IPython.display import clear_output
clear_output()
import time
time.sleep(5)

import os
os.kill(os.getpid(), 9)

In [1]:
# !edge-tts --list-voices

In [2]:
#@title Utils
def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"

from googletrans import Translator

def translate_text(text, Language):
    if Language == "English" :
      target_language='en'
    if Language == "Hindi":
      target_language='hi'
    if Language == "Bengali":
      target_language='bn'
    translator = Translator()
    translation = translator.translate(text, dest=target_language)
    t_text=translation.text
    if Language == "English" :
      return t_text
    if Language == "Hindi" or Language == "Bengali":
      return t_text.replace(".","।")

def make_chunks(input_text, language):
    if language == "English":
        temp_list = input_text.strip().split(".")
        filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
        if temp_list[-1].strip():
            filtered_list.append(temp_list[-1].strip())
        return filtered_list

    if language == "Hindi" or language == "Bengali":
        temp_list = input_text.strip().split("।")
        filtered_list = [element.strip() + '।' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
        if temp_list[-1].strip():
            filtered_list.append(temp_list[-1].strip())
        return filtered_list


import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # Initialize an empty AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through each audio file path
    for audio_path in audio_paths:
        # Load the audio file using Pydub
        audio = AudioSegment.from_file(audio_path)

        # Append the current audio file to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output path
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'
      print(edge_command)
      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path


In [3]:
text = 'Hi how are you ?'  # @param {type: "string"}
Language = "English" # @param ['English', 'Hindi', 'Bengali']

Gender = "Female"# @param ['Male', 'Female']
speed = 1  # @param {type: "number"}

translate_text_flag  = True # @param {type:"boolean"}
long_sentence = False # @param {type:"boolean"}
save_path = '/content/edge.wav'  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    voice_name="en-US-ChristopherNeural"
  if Gender=="Female":
    voice_name="en-US-AriaNeural"
if Language == "Hindi":
  if Gender=="Male":
    voice_name="hi-IN-MadhurNeural"
  if Gender=="Female":
    voice_name="hi-IN-SwaraNeural"
if Language == "Bengali":
  if Gender=="Male":
    voice_name="bn-IN-BashkarNeural"
  if Gender=="Female":
    voice_name="bn-BD-NabanitaNeural"
if translate_text_flag:
  input_text=translate_text(text, Language)
  print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
from IPython.display import clear_output
clear_output()
from IPython.display import Audio
Audio(edge_save_path, autoplay=True)

In [ ]:
#@title Download edge voice

from google.colab import files
files.download(edge_save_path)